# Political ideology on Kickstarter 




In [2]:
import docx2txt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import fasttext
from keras.preprocessing import text

Using TensorFlow backend.


In [2]:
# This is the path to all kickstarter .docx files.
Rootdir = '/Users/aaorsi/Dropbox/Amanda_y_Alvaro/PythonStuff_Political/1000_Kickstarter/' 

#Retrieve all text from the files

entries = []
names = []

directory = os.fsencode(Rootdir)

iff = 0
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    if filename.endswith(".docx"): 
        f_entry = docx2txt.process("%s/%s"%(Rootdir,filename))
        entries.append(text.text_to_word_sequence(f_entry, lower=True, split=' '))
        names.append(os.path.splitext(os.path.basename(filename))[0])
        print ('.', end='' if iff%100 != 0 else '\n')
        iff += 1


.
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
....................................................................................................
.........................................................................................

In [5]:
# Load training dataset

dset = 'forum_sample.csv'

forum = pd.read_csv(dset)


forum['userPolitics'] = forum['userPolitics'].replace(
    "<bound method SelectorList.extract of \[<Selector xpath='div\[3\]\/div\[2\]\/a\[2\]\/text\(\)' data='",
    '', regex=True)
forum['userPolitics'] = forum['userPolitics'].str.extract(r"data=' (\w+)\\n")

#sLeftRight
forum['sLeftRight'] = ''
forum.loc[forum['userPolitics'].isin(['Conservative',
            'Republican','Libertarian','Capitalist']), 'sLeftRight'] = 'Conservative'
forum.loc[forum['userPolitics'].isin(['Liberal','Progressive','Socialist',
            'Democratic','Anarchist','Green']), 'sLeftRight'] = 'Liberal' 


forum['postText'] = forum['postText'].str.strip()
forum['postText'] = forum['postText'].replace('\n','', regex=True)
forum['postText'] = forum['postText'].replace('^,','', regex=True)
forum['postText'] = forum['postText'].str.strip()
forum['postText'] = forum['postText'].replace('^,','', regex=True)
forum['postText'] = forum['postText'].replace('^\s+','', regex=True)


forum



,Unnamed: 0,postDateTime,postText,threadTitle,userName,userPolitics,userPosts,sLeftRight
0,15,2016-01-02 18:24:00,"Discuss,",Do you think Trump will ever mention the horri...,xxxxxxx,Revolutionary,"16,928",
1,16,2016-01-02 18:49:00,Not really. I'm pretty sure Trump follows the...,Do you think Trump will ever mention the horri...,TotallyRad,Liberal,"9,903",Liberal
2,17,2016-01-02 19:48:00,"Ohhhhhhhh....if you look at that list on WIKI,...",Do you think Trump will ever mention the horri...,xxxxxxx,Revolutionary,"16,928",
3,18,2016-01-02 20:29:00,No denial that Bill Clinton is sleazy. Honest...,Do you think Trump will ever mention the horri...,TotallyRad,Liberal,"9,903",Liberal
4,19,2016-01-02 22:23:00,Truman was total class and gentleman. Ike was ...,Do you think Trump will ever mention the horri...,xxxxxxx,Revolutionary,"16,928",
5,20,2016-01-02 23:07:00,Nixon for all his faults was a very good admin...,Do you think Trump will ever mention the horri...,TotallyRad,Liberal,"9,903",Liberal
6,21,2016-02-03 18:44:00,"""The director of Islamic teaching at the mosqu...",Turns Out Obama is a Homophobe and a Misogynist,roadkill,Conservative,"33,603",Conservative
7,22,2016-02-03 18:57:00,Where are all the insane Hillary supporters on...,Turns Out Obama is a Homophobe and a Misogynist,roadkill,Conservative,"33,603",Conservative
8,23,2016-02-03 19:18:00,"Is barak the obogus a raghead? He says, ""No!""...",Turns Out Obama is a Homophobe and a Misogynist,nefarious101,NaN,"20,598",
9,24,2016-02-03 19:19:00,"Not a Clinton supporter. ,Dont be Muslim if y...",Turns Out Obama is a Homophobe and a Misogynist,JoeyBone,NaN,"16,780",


In [13]:
print (forum['postText'][999])

My agreement with a genuine conservative, you unschooled, knee jerk progressive, is clear to all who can read.,Nowhere did EITHER of us mention democrats - yet that was the first thing out of your dick sucker. ,Now, you walk it off, son - or I will give you something to cry about.


In [ ]:
model['ok']